In [1]:
pip install selenium tqdm pandas folium  openpyxl xlrd ipywidgets

In [2]:
from selenium import webdriver
from tqdm.notebook import tqdm
from tqdm import tqdm_notebook as tqdmn
import pandas as pd
import folium

In [3]:
%%HTML
<style>.dataframe th{background: rgb(63,87,124);background: linear-gradient(180deg, rgba(63,87,124,1) 0%, rgba(101,124,161,1) 100%, rgba(0,212,255,1) 100%);;
padding: 10px;font-family: monospace;font-size: 110%;color: white;border:1px dashed white;text-align:left !important;
-moz-border-radius: 3x;-webkit-border-radius: 3px;}.dataframe thead{border:none; !important;}</style>

In [4]:
pd.set_option('display.max_columns', 60)

acnc = pd.read_excel('jkas_omp.xlsx', keep_default_na=False)

In [5]:
acnc.head()

,coordinate_id,lokasi,parlimen,taman,parlimen_subarea,jumlah_unit_premis,kekerapan_kutipan_sisa_domestik,kekerapan_kutipan_sampah_pukal,kekerapan_kutipan_sampah_haram,ukuran_panjang_sapuan_jalan,ukuran_panjang_sapuan_TPKK,ukuran_panjang_sapuan_kewlapangparkir,ukuran_panjang_jejantas_sapuan,ukuran_panjang_jejantas_cucian,ukuran_panjang_cucian_siarkaki,ukuran_panjang_cucian_siarkaki_berbumbung,ukuran_panjang_cucian_slesenbaslteksi,ukuran_panjang_cucilongkan,ukuran_panjang_potongrumput,ukuran_panjang_sampahkebun,catatan,rujuken_tarikh_serahan,tarikh_semakandi_lapangant_keadeansemata_ada,tarikh_semakandi_lapangant_keadeansemata_tiada
0,1,Pasar Malam Taman Bringin,Batu,TAMAN BERINGIN,,,SKS,,,SKS,,,,,,,,,,,,(76)dlm.DBKL/JKAS(BAS)/7/1/7-4 Jld 2,13 Disember 2017,
1,2,Sek. Rendah Agama Dzar Al-Ghifari Taman Beringin,Batu,TAMAN BERINGIN,BT1-1,1,SKS,,,,,,,,,,,,,,MGB660 X 2,(40)dlm.DBKL/JKAS(BAS)/7/1/7-4,1 Ogos 2014,
2,3,Jalan Miri PPRT Taman Beringin 1,Batu,TAMAN BERINGIN,BT1-1,,7M,-,-,-,,,,,,,,-,,-,6 SWB,,Mei 2017,
3,4,Jalan Miri Gerai,Batu,TAMAN BERINGIN,BT1-1,,6M,Isnin,-,6M,,,,,,,,1S,,Isnin,MGB 240L,,Mei 2017,
4,5,Jalan Miri Kedai,Batu,TAMAN BERINGIN,BT1-1,,6M,Isnin,-,6M,,,,,,,,1S,,Isnin,MGB 240L,,Mei 2017,


In [6]:
acnc['full_address'] = acnc['lokasi'].str.cat( acnc[['taman', 'parlimen']], sep=',')

In [7]:
acnc.full_address

0           Pasar Malam Taman Bringin,TAMAN BERINGIN,Batu
1       Sek. Rendah Agama Dzar Al-Ghifari Taman Bering...
2       Jalan Miri PPRT Taman Beringin 1,TAMAN BERINGI...
3                    Jalan Miri Gerai,TAMAN BERINGIN,Batu
4                    Jalan Miri Kedai,TAMAN BERINGIN,Batu
                              ...                        
7554    Lebuhraya KL-Seremban (Terowong Smart hingga s...
7555    PPA1M Bukit Jalil,  No 12 Lebuhraya Bukit Jali...
7556    Dari Susur Taman Salak Selatan Hingga Ke Sri P...
7557              Jalan Mas 2,TAMAN SALAK SELATAN,Seputeh
7558    Residensi Jalilmas No. 75 Lebuhraya Bukit Jali...
Name: full_address, Length: 7559, dtype: object

In [8]:
acnc['url'] = ['https://www.google.com/maps/search/' + i for i in acnc['full_address'] ]

In [9]:
acnc.url

0       https://www.google.com/maps/search/Pasar Malam...
1       https://www.google.com/maps/search/Sek. Rendah...
2       https://www.google.com/maps/search/Jalan Miri ...
3       https://www.google.com/maps/search/Jalan Miri ...
4       https://www.google.com/maps/search/Jalan Miri ...
                              ...                        
7554    https://www.google.com/maps/search/Lebuhraya K...
7555    https://www.google.com/maps/search/PPA1M Bukit...
7556    https://www.google.com/maps/search/Dari Susur ...
7557    https://www.google.com/maps/search/Jalan Mas 2...
7558    https://www.google.com/maps/search/Residensi J...
Name: url, Length: 7559, dtype: object

In [13]:
url_with_coordinates = []

option = webdriver.ChromeOptions()
prefs = {'profile.default_content_setting_values': {'images':2, 'javascript':2}}
option.add_experimental_option('prefs', prefs)

driver = webdriver.Chrome("C:\\chromedriver.exe", options=option)

for url in tqdm(acnc.url, leave=False):
    driver.get(url)
    url_with_coordinates.append(driver.find_element_by_css_selector('meta[itemprop=image]').get_attribute('content'))

driver.close()

import csv

with open('url_with_coordinates.csv', 'w') as file:
    wr = csv.writer(file)
    wr.writerow(url_with_coordinates)

  0%|          | 0/7559 [00:00<?, ?it/s]

In [14]:
import csv

with open('url_with_coordinates.csv', 'r') as f:
    reader = csv.reader(f, delimiter=',')
    for i in reader:
        url_with_coordinates = i
        break

In [15]:
acnc['url_with_coordinates'] = url_with_coordinates

In [16]:
acnc[~acnc.url_with_coordinates.str.contains('&zoom=')]

,coordinate_id,lokasi,parlimen,taman,parlimen_subarea,jumlah_unit_premis,kekerapan_kutipan_sisa_domestik,kekerapan_kutipan_sampah_pukal,kekerapan_kutipan_sampah_haram,ukuran_panjang_sapuan_jalan,ukuran_panjang_sapuan_TPKK,ukuran_panjang_sapuan_kewlapangparkir,ukuran_panjang_jejantas_sapuan,ukuran_panjang_jejantas_cucian,ukuran_panjang_cucian_siarkaki,ukuran_panjang_cucian_siarkaki_berbumbung,ukuran_panjang_cucian_slesenbaslteksi,ukuran_panjang_cucilongkan,ukuran_panjang_potongrumput,ukuran_panjang_sampahkebun,catatan,rujuken_tarikh_serahan,tarikh_semakandi_lapangant_keadeansemata_ada,tarikh_semakandi_lapangant_keadeansemata_tiada,full_address,url,url_with_coordinates
24,25,Jalan 1/16E,Batu,TAMAN NANYANG,BT1-1,,SKS,Isnin,-,SKS,,,,,,,,1S,,Isnin,,,Mei 2017,,"Jalan 1/16E,TAMAN NANYANG,Batu",https://www.google.com/maps/search/Jalan 1/16E...,//www.gstatic.com/images/branding/product/1x/m...
25,26,Jalan 2/16E,Batu,TAMAN NANYANG,BT1-1,,SKS,Isnin,-,SKS,,,,,,,,1S,,Isnin,,,Mei 2017,,"Jalan 2/16E,TAMAN NANYANG,Batu",https://www.google.com/maps/search/Jalan 2/16E...,//www.gstatic.com/images/branding/product/1x/m...
26,27,Jalan 3/16E,Batu,TAMAN NANYANG,BT1-1,,SKS,Isnin,-,SKS,,,,,,,,1S,,Isnin,,,Mei 2017,,"Jalan 3/16E,TAMAN NANYANG,Batu",https://www.google.com/maps/search/Jalan 3/16E...,//www.gstatic.com/images/branding/product/1x/m...
27,28,Jalan 4/16E,Batu,TAMAN NANYANG,BT1-1,,SKS,Isnin,-,SKS,,,,,,,,1S,,Isnin,,,Mei 2017,,"Jalan 4/16E,TAMAN NANYANG,Batu",https://www.google.com/maps/search/Jalan 4/16E...,//www.gstatic.com/images/branding/product/1x/m...
64,65,Jalan 1/3C,Batu,TAMAN WAHYU,BT1-1,,6M,Isnin,-,IRJ,,,,,,,,1S,,Selasa,,,Mei 2017,,"Jalan 1/3C,TAMAN WAHYU,Batu","https://www.google.com/maps/search/Jalan 1/3C,...",//www.gstatic.com/images/branding/product/1x/m...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7466,7467,"Jalan 3/149A No. 227,229 & 231 No. 44,46 & 48J...",Seputeh,TAMAN BUKIT INTAN,SPTH8-6,,IRJ,Jumaat,-,IRJ,,,,,,,,1S,,Rabu,,,,,"Jalan 3/149A No. 227,229 & 231 No. 44,46 & 48J...",https://www.google.com/maps/search/Jalan 3/149...,//www.gstatic.com/images/branding/product/1x/m...
7467,7468,Jalan 4/149A,Seputeh,TAMAN BUKIT INTAN,SPTH8-6,,IRJ,Jumaat,-,IRJ,,,,,,,,1S,,Rabu,,,,,"Jalan 4/149A,TAMAN BUKIT INTAN,Seputeh",https://www.google.com/maps/search/Jalan 4/149...,//www.gstatic.com/images/branding/product/1x/m...
7469,7470,4 Unit rumah berangkai 3 tingkat di atas lot n...,Seputeh,TAMAN BUKIT INTAN,SPTH8-6,4,IRJ,,,,,,,,,,,,,,MGB120L X 4 Unit,(13)dlm.DBKL/JKAS(BAS)/7/1/9-3,1-Sep-18,,4 Unit rumah berangkai 3 tingkat di atas lot n...,https://www.google.com/maps/search/4 Unit ruma...,//www.gstatic.com/images/branding/product/1x/m...
7470,7471,"No.45, 47, 49 Jalan 4/149A, Taman Bukit Intan",Seputeh,TAMAN BUKIT INTAN,SPTH8-6,3,SKS,,,,,,,,,,,,,,MGB120L x 3 Unit,(11)dlm.DBKL/JKAS(BAS)/7/1/7-9 Jld 3,1-Sep-18,,"No.45, 47, 49 Jalan 4/149A, Taman Bukit Intan,...","https://www.google.com/maps/search/No.45, 47, ...",//www.gstatic.com/images/branding/product/1x/m...


In [17]:
list(acnc[~acnc.url_with_coordinates.str.contains('&zoom=')].url_with_coordinates)

['//www.gstatic.com/images/branding/product/1x/maps_round_512dp.png',
 '//www.gstatic.com/images/branding/product/1x/maps_round_512dp.png',
 '//www.gstatic.com/images/branding/product/1x/maps_round_512dp.png',
 '//www.gstatic.com/images/branding/product/1x/maps_round_512dp.png',
 '//www.gstatic.com/images/branding/product/1x/maps_round_512dp.png',
 '//www.gstatic.com/images/branding/product/1x/maps_round_512dp.png',
 '//www.gstatic.com/images/branding/product/1x/maps_round_512dp.png',
 '//www.gstatic.com/images/branding/product/1x/maps_round_512dp.png',
 '//www.gstatic.com/images/branding/product/1x/maps_round_512dp.png',
 '//www.gstatic.com/images/branding/product/1x/maps_round_512dp.png',
 '//www.gstatic.com/images/branding/product/1x/maps_round_512dp.png',
 '//www.gstatic.com/images/branding/product/1x/maps_round_512dp.png',
 '//www.gstatic.com/images/branding/product/1x/maps_round_512dp.png',
 '//www.gstatic.com/images/branding/product/1x/maps_round_512dp.png',
 '//www.gstatic.com/

In [19]:
acnc = acnc[acnc.url_with_coordinates.str.contains('&zoom=')].copy()

In [20]:
acnc['lat'] = [ url.split('?center=')[1].split('&zoom=')[0].split('%2C')[0] for url in acnc['url_with_coordinates'] ]
acnc['long'] = [url.split('?center=')[1].split('&zoom=')[0].split('%2C')[1] for url in acnc['url_with_coordinates'] ]

In [21]:
acnc.head()


,coordinate_id,lokasi,parlimen,taman,parlimen_subarea,jumlah_unit_premis,kekerapan_kutipan_sisa_domestik,kekerapan_kutipan_sampah_pukal,kekerapan_kutipan_sampah_haram,ukuran_panjang_sapuan_jalan,ukuran_panjang_sapuan_TPKK,ukuran_panjang_sapuan_kewlapangparkir,ukuran_panjang_jejantas_sapuan,ukuran_panjang_jejantas_cucian,ukuran_panjang_cucian_siarkaki,ukuran_panjang_cucian_siarkaki_berbumbung,ukuran_panjang_cucian_slesenbaslteksi,ukuran_panjang_cucilongkan,ukuran_panjang_potongrumput,ukuran_panjang_sampahkebun,catatan,rujuken_tarikh_serahan,tarikh_semakandi_lapangant_keadeansemata_ada,tarikh_semakandi_lapangant_keadeansemata_tiada,full_address,url,url_with_coordinates,lat,long
0,1,Pasar Malam Taman Bringin,Batu,TAMAN BERINGIN,,,SKS,,,SKS,,,,,,,,,,,,(76)dlm.DBKL/JKAS(BAS)/7/1/7-4 Jld 2,13 Disember 2017,,"Pasar Malam Taman Bringin,TAMAN BERINGIN,Batu",https://www.google.com/maps/search/Pasar Malam...,https://maps.google.com/maps/api/staticmap?cen...,-2.67925065,108.4562175
1,2,Sek. Rendah Agama Dzar Al-Ghifari Taman Beringin,Batu,TAMAN BERINGIN,BT1-1,1,SKS,,,,,,,,,,,,,,MGB660 X 2,(40)dlm.DBKL/JKAS(BAS)/7/1/7-4,1 Ogos 2014,,Sek. Rendah Agama Dzar Al-Ghifari Taman Bering...,https://www.google.com/maps/search/Sek. Rendah...,https://maps.google.com/maps/api/staticmap?cen...,3.2149315,101.6646907
2,3,Jalan Miri PPRT Taman Beringin 1,Batu,TAMAN BERINGIN,BT1-1,,7M,-,-,-,,,,,,,,-,,-,6 SWB,,Mei 2017,,"Jalan Miri PPRT Taman Beringin 1,TAMAN BERINGI...",https://www.google.com/maps/search/Jalan Miri ...,https://maps.google.com/maps/api/staticmap?cen...,3.2206674,101.66474145
3,4,Jalan Miri Gerai,Batu,TAMAN BERINGIN,BT1-1,,6M,Isnin,-,6M,,,,,,,,1S,,Isnin,MGB 240L,,Mei 2017,,"Jalan Miri Gerai,TAMAN BERINGIN,Batu",https://www.google.com/maps/search/Jalan Miri ...,https://maps.google.com/maps/api/staticmap?cen...,3.23893485,101.6811754
4,5,Jalan Miri Kedai,Batu,TAMAN BERINGIN,BT1-1,,6M,Isnin,-,6M,,,,,,,,1S,,Isnin,MGB 240L,,Mei 2017,,"Jalan Miri Kedai,TAMAN BERINGIN,Batu",https://www.google.com/maps/search/Jalan Miri ...,https://maps.google.com/maps/api/staticmap?cen...,3.21980565,101.66474145


In [22]:
acnc.to_csv("jkas_omp.csv", index=False)